# Semanal Latinoamerica y USA importaciones semana 43 26-Octubre a 1 Noviembre

In [84]:
import pandas as pd
master=pd.read_csv(r"C:\Users\artuv\OneDrive\Bases_Chile_y_UvasPeru\Master.csv")
current_week=43

In [85]:
latam=master[(master['Region']=='LATIN AMERICA')& (master['ETD Week'].str.contains('2025'))]
latam=latam.rename(columns={'Dep_Week': 'Departure Week'}) 

In [ ]:
line_to_run= '''RUN THIS LINE  in the terminal #python -m nbconvert --to html --no-input "Weekly.ipynb" --log-level CRITICAL
#python -m nbconvert --to webpdf --no-input "Weekly.ipynb" --TagRemovePreprocessor.enabled=True --PDFExporter.latex_command="['pdflatex', '{filename}', '--landscape']" '''
import warnings
warnings.filterwarnings('ignore')

### Polar Fruit Exports most recent

In [87]:
polar=master[((master['Exporter']=='POLAR FRUIT INT')| (master['Exporter']=='POLAR FRUIT INTERNATIONAL SAC'))& master['Year Country'].str.contains('2025')]
polar=polar.rename(columns={'Dep_Week': 'Departure Week','Year Country':'Exporting Country'})
polar=polar.sort_values(by=['Departure Week','Boxes']) # Sort polar before pivoting
polar_recent=polar[polar['Departure Week']==current_week]
polar_pivot=polar_recent.pivot_table(index=['Departure Week','Country','Importer'],
                                     columns=['Exporting Country','Specie','Variety'],
                                     values='Boxes',aggfunc='sum',margins=True).fillna(0)
country_rn={'2025-CHILE':'Chile','2025-PERU':'Peru'}
polar_pivot.rename(columns=country_rn,inplace=True)
polar_pivot=polar_pivot.style.format("{:,.0f}")
polar_pivot

## Latest Week in Imports from Chile and Peru  to the USA week 43

In [88]:
usa25=master[(master['Country']=='U.S.A.')& master['Year Country'].str.contains('2025')]
usa25=usa25.rename(columns={'Dep_Week': 'Departure Week','Year Country':'Exporting Country'})
usa_recent=usa25[usa25['Departure Week']==current_week]
usa_pivot=usa_recent.pivot_table(index=['Market','Importer','Exporter'],
                                 columns=['Exporting Country', 'Specie', 'Variety group'],
                                 values='Boxes',aggfunc='sum', margins=True).fillna(0)
usa_rename={'2025-CHILE':'Chile','2025-PERU':'Peru'}
specie_totals = usa_pivot.groupby(level=[0, 1], axis=1).sum()
specie_totals.columns = pd.MultiIndex.from_tuples(
        [(c[0], c[1], 'TOTAL') for c in specie_totals.columns],
        names=usa_pivot.columns.names
    )
usa_pivot = pd.concat([usa_pivot, specie_totals], axis=1).sort_index(axis=1)
usa_pivot.rename(columns=usa_rename,inplace=True)
usa_pivot=usa_pivot.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
    ]).set_properties(**{'text-align': 'center'})
usa_pivot

In [89]:
countries = [
    'COLOMBIA', 'CHILE', 'COSTA RICA', 'ECUADOR', 
    'EL SALVADOR', 'HONDURAS', 'GUATEMALA','MEXICO', 
    'PANAMA', 'PERU', 'VENEZUELA'
]
country_pivots = {}
for country in countries:
    df_country = latam[latam['Country'] == country]
    df_country_new=df_country[df_country['Departure Week']==current_week]
    df_country_new=df_country_new.rename(columns={'Year Country':'Exporting Country'})
    pivot = (
        df_country_new.pivot_table(
            index=['Departure Week','Importer','Exporter',],
            columns=['Exporting Country', 'Specie', 'Variety group'],
            values='Boxes',
            aggfunc='sum',
            margins=True
        )
        .fillna(0)
    )
    yc_remap={'2025-CHILE':'Chile','2025-PERU':'Peru'}
    specie_totals = pivot.groupby(level=[0, 1], axis=1).sum()
    specie_totals.columns = pd.MultiIndex.from_tuples(
        [(c[0], c[1], 'TOTAL') for c in specie_totals.columns],
        names=pivot.columns.names
    )
    pivot = pd.concat([pivot, specie_totals], axis=1).sort_index(axis=1)
    pivot.rename(columns=yc_remap,inplace=True)
    pivot=pivot.style.format("{:,}").set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'center')]},
    {'selector': 'td', 'props': [('text-align', 'center')]}
    ]).set_properties(**{'text-align': 'center'})

    country_pivots[country] = pivot

   <p style="page-break-after:always;"></p>

## Latest Week in Imports from Chile and or  Peru for Latin America by Country

### Colombia

In [90]:
colombia_pivot = country_pivots['COLOMBIA']
colombia_pivot

### Chile

In [91]:
chile_pivot = country_pivots['CHILE']
chile_pivot

Departure Week,Importer,Exporter


<p style="page-break-after:always;"></p>

### Costa Rica

In [92]:
costa_rica_pivot = country_pivots['COSTA RICA']
costa_rica_pivot

### Ecuador

In [93]:
ecuador_pivot = country_pivots['ECUADOR']
ecuador_pivot

### El Salvador

In [94]:
el_salvador_pivot = country_pivots['EL SALVADOR']
el_salvador_pivot

### Guatemala

In [95]:
guatemala_pivot = country_pivots['GUATEMALA']
guatemala_pivot

<p style="page-break-after:always;"></p>

### Honduras

In [96]:
honduras=country_pivots['HONDURAS']
honduras

### Mexico

In [97]:
mx=country_pivots['MEXICO']
mx

   <p style="page-break-after:always;"></p>

### Panama

In [98]:
panama_pivot = country_pivots['PANAMA']
panama_pivot

### Peru

In [99]:
peru_pivot = country_pivots['PERU']
peru_pivot

<p style="page-break-after:always;"></p>

### Venezuela

In [100]:
venezuela_pivot = country_pivots['VENEZUELA']
venezuela_pivot